In [5]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.patches as mpatches
sns.set_style("ticks")
sns.set_context("poster")
from attrdict import AttrDict

from data import loaders

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})


In [6]:
benchmark =  AttrDict({
    'name': 'synthetic_1D_sinusoidal',
    'NS': 25000,
    'config': {
        'Y0_PDF': 'piecewise_sinusoid',
        'Y1_PDF': 'low_base_rate_sinusoid',
        'PI_PDF': 'linear'
        }
})

error_params = AttrDict({
    'alpha_0': 0.3,
    'alpha_1': 0,
    'beta_0': 0.1,
    'beta_1': 0 
})

X_train, X_test, Y_train, Y_test = loaders.get_benchmark(benchmark, error_params, 30000)


In [7]:
x = X_test.T.to_numpy().squeeze()
inds = np.argsort(x)
ys0sorted = Y_test['pYS_0'].to_numpy()[inds]
ys1sorted = Y_test['pYS_1'].to_numpy()[inds]
pd = Y_test['pD'].to_numpy()[inds]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(4, 8), sharex=True)
axes[0].plot(x[inds], ys0sorted, label='$\eta^*_0(x)$', color='darkblue')
axes[0].plot(x[inds], 1-pd, label='$1-\pi(x)$', color='black', linestyle='dashed')
axes[0].set_ylabel('$P$')
axes[0].legend(bbox_to_anchor=(1, 0.8))


axes[1].plot(x[inds], ys1sorted, label='$\eta^*_1(x)$', color='orange')
axes[1].plot(x[inds], pd, label='$\pi(x)$', color='black', linestyle='dotted')
axes[1].set_xlabel('$X$')
axes[1].legend(bbox_to_anchor=(1, 0.8))

plt.savefig('figs/1D_synthetic_setup.pgf', bbox_inches = 'tight', dpi=500)


RuntimeError: pdflatex not found.  Install it or change rcParams['pgf.texsystem'] to an available TeX implementation.

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x12b582ca0> (for post_execute):


RuntimeError: pdflatex not found.  Install it or change rcParams['pgf.texsystem'] to an available TeX implementation.

# 2D Linear Seperable

In [ ]:
syn2d = {
    'name': 'synthetic_2D_linsep',
    'NS': 100,
    'config': {
        'Y0_PDF': '2D_linsep_baseline',
        'Y1_PDF': '2D_linsep_intervention',
        'PI_PDF': '6cov_linsep'
    }
}

param_configs = [{
    'alpha_0': 0,
    'alpha_1': 0,
    'beta_0': 0,
    'beta_1': 0 
}]


X, Y = loaders.load_dataset(syn2d, param_configs[0])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5), sharey=True)

neg_baseline_X = X[Y['YS_0'] == 0]
pos_baseline_X = X[Y['YS_0'] == 1]
neg_baseline_Y = Y[Y['YS_0'] == 0]
pos_baseline_Y = Y[Y['YS_0'] == 1]

axes[0].axline((0, 0), slope=1, color='grey', linestyle='--')
axes[0].scatter(neg_baseline_X[[0]], neg_baseline_X[[1]], color='red', marker='o',  s=(18*(3.3*(1-neg_baseline_Y['pD']))**3.2))
axes[0].scatter(pos_baseline_X[[0]], pos_baseline_X[[1]], color='blue',marker='o', s=(18*(3.3*(1-pos_baseline_Y['pD']))**3.2))
axes[0].set_title('$Y^*_0$', fontsize=18)
axes[0].text(0, -.07, '$\eta^*_0 = x_2 > x_1$', fontsize = 16)
axes[0].set_ylim([-.1, 1.05])
axes[0].set_ylabel('$X_2$', fontsize=16)
axes[0].set_xlim([-.05, 1.05])
axes[0].set_xlabel('$X_1$', fontsize=16)
axes[0].set_yticks([0, .5, 1])

# plt.figure()
neg_intervention_X = X[Y['YS_1'] == 0]
pos_intervention_X = X[Y['YS_1'] == 1]
neg_intervention_Y = Y[Y['YS_1'] == 0]
pos_intervention_Y = Y[Y['YS_1'] == 1]
axes[1].axline((1, 0), slope=-1, color='grey', linestyle='--')
axes[1].scatter(neg_intervention_X[[0]], neg_intervention_X[[1]], color='red', marker='o', s=18*(5*neg_intervention_Y['pD'])**3)
axes[1].scatter(pos_intervention_X[[0]], pos_intervention_X[[1]], color='blue', marker='o', s=18*(5*pos_intervention_Y['pD'])**3)
axes[1].set_title('$Y^*_1$', fontsize=18)
axes[1].text(0, -.07, '$\eta^*_1 = x_2 > 1 - x_1$', fontsize = 16)
axes[1].set_ylim([-.1, 1.05])
axes[1].set_xlim([-.05, 1.05])
axes[1].set_xlabel('$X_1$', fontsize=16)

X_neg_obs = X[((Y['D'] == 0) & (Y['YS_0'] == 0)) | ((Y['D'] == 1) & (Y['YS_1'] == 0))]
X_pos_obs = X[((Y['D'] == 0) & (Y['YS_0'] == 1)) | ((Y['D'] == 1) & (Y['YS_1'] == 1))]

Y_neg_neg = Y[((Y['D'] == 0) & (Y['YS_0'] == 0)) | ((Y['D'] == 1) & (Y['YS_1'] == 0))]
Y_pos_obs = Y[((Y['D'] == 0) & (Y['YS_0'] == 1)) | ((Y['D'] == 1) & (Y['YS_1'] == 1))]

axes[2].axline((0, 0), slope=1, color='grey', linestyle=':')
axes[2].scatter(X_neg_obs[[0]], X_neg_obs[[1]], color='red', marker='o', s=(15*(3*(1-Y_neg_neg['pD']))**3))
axes[2].scatter(X_pos_obs[[0]], X_pos_obs[[1]], color='blue',marker='o',s=(15*(3*(1-Y_pos_obs['pD']))**3))

# axes[2].scatter(neg_intervention_X[[0]], neg_intervention_X[[1]], color='red', marker='x', s=18*(5*neg_intervention_Y['pD'])**3)
# axes[2].scatter(pos_intervention_X[[0]], pos_intervention_X[[1]], color='blue', marker='x', s=18*(5*pos_intervention_Y['pD'])**3)
axes[2].set_title('$Y^*$', fontsize=18)
axes[2].set_ylim([-.1, 1.05])
axes[2].set_xlim([-.05, 1.05])
axes[2].set_xlabel('$X_1$', fontsize=16)


plt.savefig('figs/2D_synthetic_env.png', dpi=400, bbox_inches='tight',)
